# **<span> <font color='#b8e1f1'>VALE - SOFI - VIC</font> </span>** <font color='#p8e2l1'>- Versión 12</font>


# Clase GestorJSON (Gestor de Archivos)

In [65]:
import json

class GestorJSON:
    """ Clase para gestionar archivos en formato JSON que contienen información de estudiantes. Permite abrir, guardar, agregar y eliminar registros. """
    
    @staticmethod
    def abrir_json(ruta:str) -> list: 
        """Abre un archivo JSON y retorna su contenido como lista de diccionarios.

        Parámetro:
            ruta (str): Ruta del archivo JSON a abrir.

        Retorna:
            list: Lista de diccionarios almacenados en el archivo."""
        
        try:
            with open(ruta, 'r', encoding='utf-8') as archivo:
                datos = json.load(archivo)
        
        except FileNotFoundError:
            print(f"Error: El archivo {ruta} no existe.")        
        
        except json.JSONDecodeError:
            print("Error: El archivo no contiene un JSON válido.")
            
        except Exception as e:
            print(f"Error inesperado al abrir el archivo: {e}")
        
        else:
            return datos

    @staticmethod
    def guardar_json(ruta:str, datos:list) -> None:
        """Guarda una lista de diccionarios en un archivo JSON.

        Parámetros:
            ruta (str): Ruta del archivo JSON donde se guardarán los datos.
            datos (list): Lista de diccionarios con los datos a guardar."""
        
        try:
            with open(ruta, mode="w", encoding="utf-8") as archivo:
                json.dump(datos, archivo, indent= 4, ensure_ascii=False)
        
        except TypeError:
            print("Error: Los datos a guardar no son serializables como JSON.")
        
        except IOError:
            print(f"Error: No se pudo abrir el archivo {ruta} para escribir.")
        
        except Exception as e:
            print(f"Error inesperado al guardar el archivo: {e}")

    @staticmethod
    def agregar_estudiante(ruta:str, nuevo_estudiante:dict) -> str:
        """Agrega un nuevo estudiante al archivo JSON.

        Parámetros:
            ruta (str): Ruta del archivo JSON donde se almacena la base de datos.
            nuevo_estudiante (dict): Diccionario con los datos del estudiante a agregar."""
        
        datos_estudiantes = GestorJSON.abrir_json(ruta)
            
        datos_estudiantes.append(nuevo_estudiante)
        
        GestorJSON.guardar_json(ruta, datos_estudiantes)
    
    @staticmethod
    def eliminar_estudiante(ruta:str, indice_estudiante: int) -> str:
        """Elimina un estudiante del archivo JSON.

        Parámetros:
            ruta (str): Ruta del archivo JSON donde se almacena la base de datos.
            indice_estudiante (int): Posición dentro del archivo JSON donde se encuentra del estudiante a eliminar."""

        datos_estudiantes = GestorJSON.abrir_json(ruta)
        
        datos_estudiantes.pop(indice_estudiante)
                
        GestorJSON.guardar_json(ruta, datos_estudiantes)
    
    @staticmethod
    def modificar_info_carrera(ruta: str, carrera: str, clave: str, nuevo_valor):
        """Modifica un valor del archivo JSON de informacion de las carreras para una carrera dada."""
        
        datos = GestorJSON.abrir_json(ruta)
       
        datos[carrera][clave] = nuevo_valor
            
        GestorJSON.guardar_json(ruta, datos)

    @staticmethod
    def agregar_carrera(ruta: str, nombre_carrera: str, datos_carrera: dict):
        """Agrega una nueva carrera al archivo JSON de informacion de las carreras."""
        
        datos = GestorJSON.abrir_json(ruta)
        
        datos[nombre_carrera] = datos_carrera
        
        GestorJSON.guardar_json(ruta, datos)

# Clase Plan de Estudio

In [66]:
class PlanEstudio:
    """ Clase que gestiona un plan de estudio a partir de un archivo JSON. """
    
    def __init__(self, ruta_carreras: str, carrera:str):
        """Inicializa el sistema con el plan de estudio de determinada carrera."""
        
        info_carreras = GestorJSON.abrir_json(ruta_carreras)
        self.__carrera = carrera
        
        if carrera not in info_carreras:
            raise ValueError(f"La carrera '{carrera}' no está registrada.")
        
        ruta_plan = info_carreras[carrera]["plan_path"]
        self.__plan = GestorJSON.abrir_json(ruta_plan)
    
    @property
    def plan(self) -> list: 
        """Obtiene el plan de estudio de la carrera."""
        return self.__plan
    
    @property
    def carrera(self) -> list:
        """Obtiene la carrera del plan de estudio."""
        return self.__carrera
    
    def ucs_por_semestre(self, semestre: int) -> list:
        """Obtiene todas las Unidades Curriculares (UC) correspondientes a un semestre específico.

        Parámetro:
            semestre (int): Número del semestre (entre 1 y 10).

        Retorna:
            list: Lista de UC del semestre especificado.
            str: Mensaje de error si el semestre no es válido."""
        
        try:
            if not (1 <= semestre <= 10):
                raise ValueError(f"El semestre {semestre} no existe en el plan de estudio.")
        
        except ValueError as e:
            print(e)
        
        else:
            return [uc for uc in self.__plan if uc["Semestre"] == semestre]
    
    def info_uc(self, codigo: str) -> dict:
        """Obtiene la información de una UC por su código.

        Parámetro:
            codigo (str): Código de la UC a buscar.

        Retorna:
            dict: Información de la UC con el código proporcionado.
            str: Mensaje de error si no se encuentra la UC."""
            
        error = self._verificar_codigo_uc(codigo)
        
        if error is not None:
            return error
        
        for uc in self.__plan:
        
            if uc["Código"] == codigo:
                return uc
    
    def previas_de_uc(self, codigo: str) -> dict:
        """Obtiene las UC previas de una UC especificada.

        Parámetro:
            codigo (str): Código de la UC para la cual se desean conocer las previas.

        Retorna:
            dict: Diccionario de las UC previas.
            str: Mensaje de error si la UC no tiene previas o no se encuentra."""
        
        error = self._verificar_codigo_uc(codigo)
        
        if error is not None:
            return error
        
        for uc in self.__plan:
            
            if uc["Código"] == codigo:
                
                if uc["Previaturas"] != {}:
                    return uc["Previaturas"]
                
                else:
                    return f"La UC '{codigo}' no tiene previas."
    
    def ucs_indice(self) -> list:
        """Obtiene un índice con los códigos y nombres de todas las UC.

        Retorna:
            list: Lista de tuplas con el código y nombre de cada UC."""
        
        return [(uc["Código"], uc["Nombre"]) for uc in self.__plan]
    
    def ucs_dependientes(self, codigo_previa: str) -> list:
        """Obtiene las UC que dependen de una UC previa específica.

        Parámetro:
            codigo_previa (str): Código de la UC previa a buscar.

        Retorna:
            list: Lista de nombres de UC dependientes de la UC previa.
            str: Mensaje de error si no se encuentra la UC previa o no tiene dependientes."""
            
        error = self._verificar_codigo_uc(codigo_previa)
        
        if error is not None:
            return error

        dependientes = [uc["Nombre"] for uc in self.__plan if codigo_previa in uc["Previaturas"]]

        if dependientes != []:
            return dependientes
        
        else:
            return f"La UC con el código '{codigo_previa}' no es previa de ninguna UC."
        
    def sumar_creditos(self, codigos: list) -> int:
        """Suma los créditos de las Unidades Curriculares (UC) cuyos códigos se pasan como lista.

        Parámetro:
            codigos (list): Lista de códigos de las UCs a sumar.

        Retorna:
            int: Suma total de créditos de las UCs."""
       
        total_creditos = 0
        
        errores = []

        for codigo in codigos:
            
            error = self._verificar_codigo_uc(codigo)
            
            if error is not None:
                errores.append(error)
            
            for uc in self.__plan:
                    
                if uc["Código"] == codigo:
                        
                    total_creditos += uc["Créditos"]
                        
                    break

        if errores != []:
            return " - ".join(errores)
        
        return total_creditos
    
    def _verificar_codigo_uc(self, codigo:str) -> None:
        """Verifica si una Unidad Curricular (UC) con el código dado existe en el plan de estudios.

        Parámetro:
            codigo (str): Código de la unidad curricular a verificar.

        Retorna:
            Optional[str]: Mensaje de error si el código no existe, o None si es válido."""        
       
        try:
            codigos_existentes = [uc["Código"] for uc in self.__plan]
        
            if codigo not in codigos_existentes:
                raise KeyError(f"No existe una UC con el código '{codigo}'.")
        
        except KeyError as e:
            print(e)
        
        else:
            return None


### Prueba de abrir el archivo.JSON con el Plan de Estudio y despues usar los metodos del PlanEstudio

In [67]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

planIBIO.sumar_creditos(["S1UC1","S1UC2","S1UC3","S1UC4","S1UC5","S2UC1","S2UC2","S2UC4","S3UC1"])

73

### Prueba Plan de Estudios

In [68]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

planIBIO.plan

[{'Código': 'S1UC1',
  'Nombre': 'Álgebra, Análisis y Geometría Analítica',
  'Semestre': 1,
  'Créditos': 8,
  'Previaturas': {}},
 {'Código': 'S1UC2',
  'Nombre': 'Mecánica, Ondas y Calor',
  'Semestre': 1,
  'Créditos': 10,
  'Previaturas': {}},
 {'Código': 'S1UC3',
  'Nombre': 'Química General e Inorgánica',
  'Semestre': 1,
  'Créditos': 9,
  'Previaturas': {}},
 {'Código': 'S1UC4',
  'Nombre': 'Taller Inicial de Tecnologías (Anual)',
  'Semestre': 1,
  'Créditos': 0,
  'Previaturas': {}},
 {'Código': 'S1UC5',
  'Nombre': 'Salud & Sociedad',
  'Semestre': 1,
  'Créditos': 8,
  'Previaturas': {}},
 {'Código': 'S2UC1',
  'Nombre': 'Números Complejos y Ecuaciones Diferenciales',
  'Semestre': 2,
  'Créditos': 8,
  'Previaturas': {'S1UC1': 'Álgebra, Análisis y Geometría Analítica'}},
 {'Código': 'S2UC2',
  'Nombre': 'Electricidad y Magnetismo',
  'Semestre': 2,
  'Créditos': 10,
  'Previaturas': {'S1UC1': 'Álgebra, Análisis y Geometría Analítica',
   'S1UC2': 'Mecánica, Ondas y Calor'

# Clase Sistema de Matriculaciones

In [69]:
class Sistema:
    """ Clase para gestionar las matriculaciones de estudiantes. """
    
    def __init__(self, ruta_carreras:str, plan: object):
        """Inicializa el sistema con un plan de estudios.

        Parámetros:
            plan (PlanEstudio): Objeto que representa el plan de estudios."""
        
        self.__plan = plan
        self.carrera = plan.carrera
        
        self.ruta_carreras= ruta_carreras
        
        self.info_carreras = GestorJSON.abrir_json(ruta_carreras)
        
        ruta_estudiantes = self.info_carreras[self.carrera]["estudiantes_path"]
        
        self.__ruta_estudiantes = ruta_estudiantes
        self.__datos_estudiantes = GestorJSON.abrir_json(ruta_estudiantes)

    @property
    def datos_estudiantes(self) -> list:
        """Retorna una lista que representa la base de datos de los estudiantes"""
        return self.__datos_estudiantes 
    
    def _verificar_secretaria(self, nombre, apellido, codigo) -> bool:
        """ """
        
        secretaria = self.info_carreras[self.carrera]["secretaria"]
        
        if secretaria["nombre"] == nombre and secretaria["apellido"] == apellido and secretaria["codigo"] == codigo:
            return True
        
        raise PermissionError("Secretaria no autorizada.")

    def _verificar_coordinadora(self, nombre, apellido, codigo) -> bool:
        """ """

        coordinadora = self.info_carreras[self.carrera]["coordinadora"]
        
        if coordinadora["nombre"] == nombre and coordinadora["apellido"] == apellido and coordinadora["codigo"] == codigo:
            return True
        
        raise PermissionError("Coordinadora no autorizada.")
    
    def lista_estudiantes(self) -> list:
        """Devuelve una lista de todos los estudiantes con su código, nombre y apellido.
        
        Retorna:
            list: Lista de tuplas (código_estudiante, nombre, apellido) de todos los estudiantes."""
        
        return [(estudiante["codigo_estudiante"], estudiante["nombre"], estudiante["apellido"]) for estudiante in self.__datos_estudiantes]

    def info_estudiante(self, identificador: str | int) -> dict:
        """Devuelve el diccionario completo del estudiante con el identificador proporcionado.
        
        Parámetro:
            identificador (str | int): Código del estudiante (str) o cédula (int).

        Retorna:
            dict o str: Diccionario con la información completa del estudiante, o un mensaje de error."""
        
        if isinstance(identificador, (str, int)):
            
            for estudiante in self.__datos_estudiantes:
                
                if estudiante["codigo_estudiante"] == identificador or estudiante["cedula"] == identificador:
                    return estudiante
           
            return f"Error: No se encontró un estudiante con el identificador '{identificador}'."
        
        return "Error: El identificador debe ser un string (código) o un entero (cédula)."

    def estado_general_estudiante(self, codigo_estudiante: str) -> dict:
        """Devuelve información sobre las unidades curriculares aprobadas, matriculadas y pendientes de un estudiante.
        
        Parámetro:
            codigo_estudiante (str): Código único del estudiante.
        
        Retorna:
            dict o str: Diccionario con el estado de las UC (Aprobadas, Matriculadas, Pendientes) o un mensaje si no se encuentra."""
        
        error = self._verificar_codigo_estudiante(codigo_estudiante)
        
        if error is not None:
            return error
        
        for estudiante in self.__datos_estudiantes:
            
            if estudiante["codigo_estudiante"] == codigo_estudiante:
                return {
                    "Estudiante": codigo_estudiante,
                    "UC Aprobadas": estudiante["uc_aprobadas"],
                    "UC Matriculadas": estudiante["uc_matriculadas"],
                    "UC Pendientes": estudiante["uc_pendientes"]
                }
                
    def clasificar_por_ingreso(self, ingreso: int) -> list:
        """Devuelve los estudiantes que ingresaron en un determinado año.
        
        Parámetro:
            ingreso (int): Año de ingreso de los estudiantes.
        
        Retorna:
            list o str: Lista de estudiantes que ingresaron en el año especificado, o un mensaje si no hay estudiantes."""
        
        resultado= []
        
        for estudiante in self.__datos_estudiantes:
            
            if estudiante["ingreso"] == ingreso:
                resultado.append({
                        "codigo_estudiante": estudiante["codigo_estudiante"],
                        "nombre": estudiante["nombre"],
                        "apellido": estudiante["apellido"],
                        "año_ingreso": ingreso
                    })
        
        if resultado == []:
            return f"No hay ningún estudiante que haya ingresado en el año {ingreso}."
        
        else:
            return resultado

    def estado_uc_general(self, codigo_uc: str) -> list:
        """Devuelve el estado de una unidad curricular (UC) en todos los estudiantes.
        
        Parámetro:
            codigo_uc (str): Código de la unidad curricular.
        
        Retorna:
            list o str: Lista de estados de la UC para cada estudiante, o un mensaje si la UC no fue cursada."""
        
        error = self.__plan._verificar_codigo_uc(codigo_uc)
        
        if error is not None:
            return error

        resultado = []
        
        for estudiante in self.__datos_estudiantes:
            estado = self._estado_uc_auxiliar(estudiante, codigo_uc)
            
            if estado is not None:
                resultado.append({
                    "codigo_estudiante": estudiante["codigo_estudiante"],
                    "nombre": estudiante["nombre"],
                    "apellido": estudiante["apellido"],
                    "estado": estado
                })
        
        if resultado == []:
            return f"La unidad curricular '{codigo_uc}' no ha sido cursada por ningún estudiante."
        
        else:    
            return resultado

    def estado_uc_estudiante(self, codigo_estudiante: str, codigo_uc: str) -> dict:
        """Devuelve el estado de una unidad curricular (UC) para un estudiante específico.
        
        Parámetros:
            codigo_estudiante (str): Código único del estudiante.
            codigo_uc (str): Código de la unidad curricular.
        
        Retorna:
            dict o str: Diccionario con el estado de la UC para el estudiante o un mensaje si no se encuentra el estudiante."""
        
        error = self._verificar_codigo_estudiante(codigo_estudiante)
        
        if error is not None:
            return error
        
        error = self.__plan._verificar_codigo_uc(codigo_uc)
        
        if error is not None:
            return error
        
        for estudiante in self.__datos_estudiantes:
            
            if estudiante["codigo_estudiante"] == codigo_estudiante:
                estado = self._estado_uc_auxiliar(estudiante, codigo_uc)
                
                return {
                    "codigo_estudiante": estudiante["codigo_estudiante"],
                    "nombre": estudiante["nombre"],
                    "apellido": estudiante["apellido"],
                    "estado": estado or "Aún no cursada"
                }

    def _estado_uc_auxiliar(self, estudiante: dict, codigo_uc: str) -> str:              #Para evitar repetir esto en los metodos "estado_uc_general" y "estado_uc_estudiante"
        """Función auxiliar para determinar el estado de una unidad curricular (UC) en un estudiante.
        
        Parámetros:
            estudiante (dict): Diccionario con la información del estudiante.
            codigo_uc (str): Código de la unidad curricular.
        
        Retorna:
            str o None: Estado de la UC para el estudiante o None si no se encuentra."""
        
        if codigo_uc in estudiante["uc_aprobadas"]:
            return "aprobada"
        
        elif codigo_uc in estudiante["uc_matriculadas"]:
            return "matriculada"
        
        elif codigo_uc in estudiante["uc_pendientes"]["examen"]:
            return "desaprobada (examen)"
        
        elif codigo_uc in estudiante["uc_pendientes"]["tutoria"]:
            return "desaprobada (tutoria)"
        
        elif codigo_uc in estudiante["uc_pendientes"]["examen_unico"]:
            return "desaprobada (examen único)"
        
        elif codigo_uc in estudiante["uc_pendientes"]["recursar"]:
            return "desaprobada (recursar)"
        
        return None
    
    def matricular_estudiante_UC(self, codigo_estudiante: str, *codigos_UC: str) -> str:
        """Matricula a un estudiante en una o más unidades curriculares (UC).
        
        Parámetros:
            codigo_estudiante (str): Código único del estudiante.
            *codigos_UC (str): Códigos de las unidades curriculares a matricular.
        
        Retorna:
            str: Mensajes de resultado por cada intento de matriculación."""
        
        error = self._verificar_codigo_estudiante(codigo_estudiante)
        
        if error is not None:
            return error

        mensajes = []

        for estudiante in self.__datos_estudiantes:
            
            if estudiante["codigo_estudiante"] == codigo_estudiante:

                for codigo in codigos_UC:

                    error = self.__plan._verificar_codigo_uc(codigo)
                    
                    if error is not None:
                        mensajes.append(f"[{codigo}] {error}")
                        continue

                    if codigo in estudiante["uc_aprobadas"]:
                        mensajes.append(f"[{codigo}] Ya fue aprobada.")
                        continue

                    if codigo in estudiante["uc_matriculadas"]:
                        mensajes.append(f"[{codigo}] Ya está matriculada.")
                        continue

                    previas = self.__plan.previas_de_uc(codigo)
                    
                    if not all(uc_previa in estudiante["uc_aprobadas"] for uc_previa in previas):
                        mensajes.append(f"[{codigo}] No se puede matricular. Faltan previas.")
                        continue
                    
                    if codigo in estudiante["uc_pendientes"]["recursar"]:
                        estudiante["uc_pendientes"]["recursar"].remove(codigo)
                            
                    estudiante["uc_matriculadas"].append(codigo)
                    
                    mensajes.append(f"[{codigo}] Matriculación exitosa.")

                break

        estudiante["uc_matriculadas"].sort(key=lambda uc: (int(uc[1]), int(uc[4:])))
        
        GestorJSON.guardar_json(self.__ruta_estudiantes, self.__datos_estudiantes)

        return " - ".join(mensajes)
    
    def desmatricular_estudiante_UC(self, codigo_estudiante: str, *codigos_UC: str) -> str:
        """Desmatricula a un estudiante de una unidad curricular (UC).
        
        Parámetros:
            codigo_estudiante (str): Código único del estudiante.
            *codigos_UC (str): Códigos de las unidades curriculares a matricular.
        
        Retorna:
            str: Mensajes de resultado por cada intento de matriculación."""
        
        error = self._verificar_codigo_estudiante(codigo_estudiante)
        
        if error is not None:
            return error

        mensajes = []

        for estudiante in self.__datos_estudiantes:
            
            if estudiante["codigo_estudiante"] == codigo_estudiante:

                for codigo in codigos_UC:

                    error = self.__plan._verificar_codigo_uc(codigo)
                    
                    if error is not None:
                        mensajes.append(f"[{codigo}] {error}")
                        continue

                    if codigo not in estudiante["uc_matriculadas"]:
                        mensajes.append(f"[{codigo}] No está matriculada.")
                        continue

                    estudiante["uc_matriculadas"].remove(codigo)
                    
                    mensajes.append(f"[{codigo}] Desmatriculación exitosa.")

                break

        GestorJSON.guardar_json(self.__ruta_estudiantes, self.__datos_estudiantes)

        return " - ".join(mensajes)
    
    def obtener_inscritos(self, codigo_UC: str, tipo: str = None) -> list:       
        """Devuelve los estudiantes inscritos en una unidad curricular (UC).
        
        Parámetros:
            codigo_UC (str): Código de la unidad curricular.
        
        Retorna:
            list: Lista de estudiantes inscritos en la UC especificada, o un mensaje si no hay ninguno o si el código es inválido."""
        
        error = self.__plan._verificar_codigo_uc(codigo_UC)
        
        if error is not None:
            return error
        
        inscritos = []

        for estudiante in self.__datos_estudiantes:
            
            if tipo is None:
            
                if codigo_UC in estudiante["uc_matriculadas"]:
                    inscritos.append({
                        "codigo_estudiante": estudiante["codigo_estudiante"],
                        "nombre": estudiante["nombre"],
                        "apellido": estudiante["apellido"]
                    })
            
            elif tipo in ["examen", "tutoria", "examen_unico"]:
                
                pendientes = estudiante["uc_pendientes"][tipo]
                
                if codigo_UC in pendientes:
                    inscritos.append({
                        "codigo_estudiante": estudiante["codigo_estudiante"],
                        "nombre": estudiante["nombre"],
                        "apellido": estudiante["apellido"]
                    })
            
            else:
                return f"Tipo '{tipo}' no reconocido. Usa 'examen', 'tutoria', 'examen_unico'."

        if inscritos == []:
            return f"No hay estudiantes inscritos en la unidad curricular '{codigo_UC}'."
        
        return inscritos
    
    def agregar_estudiante(self, codigo_estudiante: str, nombre_estudiante: str, apellido_estudiante:str, cedula:int, ingreso:int, uc_aprobadas:list = [], uc_matriculadas:list = []) -> str:
        """Agrega un nuevo estudiante a la base de datos de estudiantes.
        
        Parámetros:
            codigo_estudiante (str): Código único del estudiante.
            nombre_estudiante (str): Nombre del estudiante.
            apellido_estudiante (str): Apellido del estudiante.
            cedula (int): Cédula de identidad del estudiante.
            ingreso (int): Año de ingreso del estudiante.
            uc_aprobadas (list, opcional): Lista de unidades curriculares aprobadas (por defecto es una lista vacía).
            uc_matriculadas (list, opcional): Lista de unidades curriculares matriculadas (por defecto es una lista vacía).
        
        Retorna:
            str: Mensaje indicando si el estudiante fue agregado exitosamente o no."""
        
        if not isinstance(codigo_estudiante, str) or not isinstance(nombre_estudiante, str) or not isinstance(apellido_estudiante, str):
            return "Error: código, nombre y apellido deben ser cadenas."
        
        if not isinstance(cedula, int) or not isinstance(ingreso, int) or not (2020 <= ingreso <= 2025):
            return "Error: cédula e ingreso deben ser enteros, y el ingreso debe ser un año entre 2020 y 2025."
        
        if not isinstance(uc_aprobadas, list) or not isinstance(uc_matriculadas, list):
            return "Error: las UC aprobadas y matriculadas deben ser listas."
        
        if len(uc_aprobadas) != len(set(uc_aprobadas)):
            return "Error: No puede haber UC duplicadas en aprobadas."
        
        if len(uc_matriculadas) != len(set(uc_matriculadas)):
            return "Error: No puede haber UC duplicadas en matriculadas."
        
        if set(uc_aprobadas) & set(uc_matriculadas):
            return "Error: Hay UCs que están tanto en aprobadas como en matriculadas."

        error = self._verificar_codigo_estudiante(codigo_estudiante)
        
        if error is None:
            return f"Ya existe un estudiante con código {codigo_estudiante}. No se agregó."
        
        for estudiante in self.__datos_estudiantes:
    
            if estudiante["cedula"] == cedula:
                return f"Ya existe un estudiante con la cédula {cedula}. No se agregó."
        
        todas_uc = uc_aprobadas + uc_matriculadas
        
        for codigo_uc in todas_uc:
            
            if self.__plan._verificar_codigo_uc(codigo_uc) is not None:
                return f"Error: La unidad curricular '{codigo_uc}' no existe en el plan de estudio."

        for uc in uc_aprobadas:
            
            previas = self.__plan.previas_de_uc(uc)
            
            if previas != f"La UC '{uc}' no tiene previas.":
                
                for uc_previa in previas:
                    
                    if uc_previa not in uc_aprobadas:
                        return f"Error: La UC '{uc}' no puede ser aprobada sin tener aprobadas todas sus previas."
                    
        for uc in uc_matriculadas:
            
            previas = self.__plan.previas_de_uc(uc)
            
            if previas != f"La UC '{uc}' no tiene previas.":
                
                for uc_previa in previas:
                    
                    if uc_previa not in uc_aprobadas:
                        return f"Error: No se puede matricular en '{uc}' sin tener aprobadas todas sus previas."

        uc_aprobadas.sort(key=lambda uc: (int(uc[1]), int(uc[4:])))
        
        uc_matriculadas.sort(key=lambda uc: (int(uc[1]), int(uc[4:])))
        
        nuevo_estudiante = {
            "codigo_estudiante": codigo_estudiante,
            "nombre": nombre_estudiante,
            "apellido": apellido_estudiante,
            "cedula": cedula,
            "ingreso": ingreso,
            "uc_aprobadas": uc_aprobadas,
            "uc_matriculadas": uc_matriculadas,
            "uc_pendientes": {
                "examen": {},
                "tutoria": {},
                "examen_unico": {},
                "recursar": []
            }
        }

        GestorJSON.agregar_estudiante(self.__ruta_estudiantes, nuevo_estudiante)
    
        return f"Estudiante '{codigo_estudiante}' fue agregado correctamente."
    
    def eliminar_estudiante(self, identificador: str | int):
        """Elimina un estudiante de la base de datos de estudiantes.

        Parámetro:
            identificador (str | int): Código del estudiante (str) o cédula (int).

        Retorna:
            str: Mensaje indicando si el estudiante fue eliminado exitosamente o no."""
        
        if not isinstance(identificador, (str, int)):
            return "Error: El identificador debe ser un string (código) o un entero (cédula)."
        
        for indice_estudiante in range(len(self.__datos_estudiantes)):
            estudiante = self.__datos_estudiantes[indice_estudiante]

            if estudiante["codigo_estudiante"] == identificador or estudiante["cedula"] == identificador:
                GestorJSON.eliminar_estudiante(self.__ruta_estudiantes, indice_estudiante)
                
                return f"Estudiante con identificador '{identificador}' fue eliminado correctamente."  
            
        return f"No se encontró ningún estudiante con los datos proporcionados '{identificador}'."
    
    def _verificar_codigo_estudiante(self, codigo: str) -> None:
        """Verifica si un estudiante con un código dado existe en la base de datos de estudiantes.

        Parámetro:
            codigo (str): Código del estudiante.

        Retorna:
            None si existe, o un mensaje de error si no existe."""
        
        codigos_existentes = [estudiante["codigo_estudiante"] for estudiante in self.__datos_estudiantes]

        if codigo not in codigos_existentes:
            return f"No existe un estudiante con el código '{codigo}'."
        
        return None
    
    def inscripcion(self, codigo_estudiante: str, codigo_uc: str, estado: str) -> str:
        """Inscribe a un estudiante en un modalidad (examen, tutoría o examen único).
        
        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular a la que se desea inscribir.
            estado (str): Modalidad de inscripción ('examen', 'tutoria' o 'examen_unico').

        Retorna:
            str: Mensaje con el resultado de la inscripción (éxito o descripción del error)."""
        
        if estado not in ["examen", "tutoria", "examen_unico"]:
            return "Error: El estado debe ser 'examen', 'tutoria' o 'examen_unico'."

        error = self._verificar_codigo_estudiante(codigo_estudiante)
        
        if error is not None:
            return error
        
        error = self.__plan._verificar_codigo_uc(codigo_uc)
        
        if error is not None:
            return error

        estudiante = self.info_estudiante(codigo_estudiante)
        
        estado_actual = self._estado_uc_auxiliar(estudiante, codigo_uc)
        
        if estado_actual is not None:
            return f"Error: La UC '{codigo_uc}' ya se encuentra en estado '{estado_actual}'."

        previas = self.__plan.previas_de_uc(codigo_uc)
       
        for previa in previas.keys():
            
            if previa not in estudiante["uc_aprobadas"]:
                
                return f"Error: No tiene aprobadas todas las UCs previas para inscribirse a '{codigo_uc}'."

        estudiante["uc_pendientes"][estado][codigo_uc] = 0
        
        GestorJSON.guardar_json(self.__ruta_estudiantes, self.__datos_estudiantes)
        
        return f"Inscripción exitosa: {codigo_estudiante} en '{codigo_uc}' para estado '{estado}'."
    
    def poner_nota(self, codigo_estudiante: str, codigo_uc: str, estado: str, nota: float) -> str:
        """Registra la nota obtenida por un estudiante en un estado (examen, tutoría, examen único) y actualiza su situación académica.
        
        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular evaluada.
            estado (str): Modalidad de evaluación ('examen', 'tutoria' o 'examen_unico').
            nota (float): Nota obtenida por el estudiante.

        Retorna:
            str: Mensaje indicando si el registro fue exitoso o describiendo el error."""
        
        if estado not in ["examen", "tutoria", "examen_unico"]:
            return "Error: El estado debe ser 'examen', 'tutoria' o 'examen_unico'."

        error = self._verificar_codigo_estudiante(codigo_estudiante)
        
        if error is not None:
            return error

        error = self.__plan._verificar_codigo_uc(codigo_uc)
        
        if error is not None:
            return error

        estudiante = self.info_estudiante(codigo_estudiante)

        if codigo_uc not in estudiante["uc_pendientes"][estado]:
            return f"Error: La UC '{codigo_uc}' no está inscrita en el estado '{estado}' para el estudiante."

        if estado == "examen":
            
            if nota >= 3.0:
                del estudiante["uc_pendientes"]["examen"][codigo_uc]
               
                estudiante["uc_aprobadas"].append(codigo_uc)
                
                mensaje = "Aprobado por examen."
            
            else:
                
                estudiante["uc_pendientes"]["examen"][codigo_uc] += 1
                
                intento = estudiante["uc_pendientes"]["examen"][codigo_uc]
                
                if intento == 3:
                    decision = input(f"El estudiante {codigo_estudiante} desaprobó 3 veces la UC '{codigo_uc}'. ¿Desea rendir examen único? (si/no): ").strip().lower()
                    
                    del estudiante["uc_pendientes"]["examen"][codigo_uc]
                    
                    if decision == "si":
                        estudiante["uc_pendientes"]["examen_unico"][codigo_uc] = 1
                        
                        mensaje = "Desaprobado 3 veces. Inscripto a examen único."
                    
                    else:
                        estudiante["uc_pendientes"]["recursar"].append(codigo_uc)
                        
                        mensaje = "Desaprobado 3 veces. Debe recursar."
                
                else:
                    mensaje = f"Desaprobado. Intento {intento}/3."

        elif estado == "tutoria":
            
            if nota not in [0.0, 3.0]:
                return "Error: La nota de tutoría solo puede ser 0 o 3."
            
            if nota == 3.0:
                del estudiante["uc_pendientes"]["tutoria"][codigo_uc]
                
                self.inscripcion(codigo_estudiante, codigo_uc, "examen")
                
                mensaje = "Tutoría aprobada, habilitado a rendir el examen reglamentado."
           
            else:
                decision = input(f"El estudiante {codigo_estudiante} desaprobó la tutoría de la UC '{codigo_uc}'. ¿Desea rendir examen único? (si/no): ").strip().lower()
                
                del estudiante["uc_pendientes"]["tutoria"][codigo_uc]
                
                if decision == "si":
                    estudiante["uc_pendientes"]["examen_unico"][codigo_uc] = 0
                    
                    mensaje = "Desaprobado en tutoría. Inscripto a examen único."
                
                else:
                    estudiante["uc_pendientes"]["recursar"].append(codigo_uc)
                    
                    mensaje = "Desaprobado en tutoría. Debe recursar."

        elif estado == "examen_unico":
            del estudiante["uc_pendientes"]["examen_unico"][codigo_uc]
            
            if nota >= 4.0:
                estudiante["uc_aprobadas"].append(codigo_uc)
               
                mensaje = "Aprobado por examen único."
            
            else:
                estudiante["uc_pendientes"]["recursar"].append(codigo_uc)
               
                mensaje = "Desaprobado en examen único. Debe recursar."

        GestorJSON.guardar_json(self.__ruta_estudiantes, self.__datos_estudiantes)

        return f"Resultado registrado para {codigo_estudiante} en '{codigo_uc}': {mensaje}"
    
    def modificar_info_carrera(self, clave: str, nuevo_valor) -> str:
        """ """

        if clave not in self.info_carreras[self.carrera]:
            return f"Error: La clave '{clave}' no existe en la información de la carrera '{self.carrera}'."

        try:
            GestorJSON.modificar_info_carrera(self.ruta_carreras, self.carrera, clave, nuevo_valor)
            
            self.info_carreras = GestorJSON.abrir_json(self.ruta_carreras)
            
            return f"Modificación exitosa: '{clave}' de '{self.carrera}' actualizado."
        
        except Exception as e:
            return f"Error al modificar la información: {e}"
                
    def agregar_carrera(self, nombre_carrera: str, plan_path: str, estudiantes_path: str, nombre_secretaria: str, apellido_secretaria: str, codigo_secretaria: str, nombre_coordinadora: str, apellido_coordinadora: str, codigo_coordinadora: str) -> str:
        """
        Agrega una nueva carrera al archivo de información.

        Parámetros:
            nombre_carrera (str): Nombre de la nueva carrera.
            plan_path (str): Ruta al archivo del plan de estudios.
            estudiantes_path (str): Ruta a la base de datos de estudiantes.
            nombre_secretaria (str): Nombre de la secretaria.
            apellido_secretaria (str): Apellido de la secretaria.
            codigo_secretaria (str): Código de la secretaria.
            nombre_coordinadora (str): Nombre de la coordinadora.
            apellido_coordinadora (str): Apellido de la coordinadora.
            codigo_coordinadora (str): Código de la coordinadora.

        Retorna:
            str: Mensaje con el resultado de la operación.
        """

        if nombre_carrera in self.info_carreras:
            return f"Error: La carrera '{nombre_carrera}' ya existe."

        datos_carrera = {
            "plan_path": plan_path,
            "estudiantes_path": estudiantes_path,
            "secretaria": {
                "nombre": nombre_secretaria,
                "apellido": apellido_secretaria,
                "codigo": codigo_secretaria
            },
            "coordinadora": {
                "nombre": nombre_coordinadora,
                "apellido": apellido_coordinadora,
                "codigo": codigo_coordinadora
            }
        }

        try:
            GestorJSON.agregar_carrera(self.ruta_carreras, nombre_carrera, datos_carrera)
            return f"Carrera '{nombre_carrera}' agregada exitosamente."
        
        except Exception as e:
            return f"Error al agregar carrera: {e}"


### Prueba de abrir el archivo.JSON con la base de datos de los estudiantes y despues usar los metodos del Sistema

In [70]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

datos_estudiantes= Sistema(ruta, planIBIO)

datos_estudiantes.info_estudiante("E011")

{'codigo_estudiante': 'E011',
 'nombre': 'Valentin',
 'apellido': 'Ferrando',
 'cedula': 52953201,
 'ingreso': 2022,
 'uc_aprobadas': ['S1UC1',
  'S1UC2',
  'S1UC3',
  'S1UC4',
  'S1UC5',
  'S2UC1',
  'S2UC2',
  'S2UC3',
  'S2UC4',
  'S3UC1',
  'S3UC2',
  'S3UC3',
  'S3UC4',
  'S3UC5',
  'S4UC1',
  'S4UC2',
  'S4UC4',
  'S4UC5',
  'S5UC1',
  'S5UC2',
  'S5UC3',
  'S5UC4',
  'S6UC1',
  'S6UC2',
  'S6UC3',
  'S6UC4',
  'S6UC5',
  'S6UC6'],
 'uc_matriculadas': ['S7UC2', 'S7UC4'],
 'uc_pendientes': {'examen': {},
  'tutoria': {},
  'examen_unico': {},
  'recursar': []}}

### Prueba de inscribir a un examen y poner nota

In [71]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

sistema= Sistema(ruta, planIBIO)

sistema.obtener_inscritos("S6UC5", "examen_unico")

#sistema.inscripcion("E001", "S3UC5", "tutoria")

#sistema.poner_nota("E001", "S3UC5", "tutoria", 0)

"No hay estudiantes inscritos en la unidad curricular 'S6UC5'."

### Prueba de abrir matricular/desmatricular una o más UCs de un estudiante

In [72]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

sistema= Sistema(ruta, planIBIO)

#sistema.info_estudiante("E011")
#print(planIBIO.ucs_por_semestre(7))

#sistema.matricular_estudiante_UC("E011", "S7UC3", "S7UC5", "S7UC1", "S7UC2", "S7UC4")
#sistema.matricular_estudiante_UC("E011", "S7UC1", "S6UC2", "S5UC3", "S7UC44", "S9UC1", "S7UC3")       #Con errores a proposito

#sistema.desmatricular_estudiante_UC("E011", "S7UC3", "S7UC1", "S7UC4")
#sistema.desmatricular_estudiante_UC("E011", "S7UC1","S7UC12", "S7UC3", "S1UC1", "S7UC4", "S7UC5", "S8UC1")    #Con errores a proposito

sistema.info_estudiante("E011")

{'codigo_estudiante': 'E011',
 'nombre': 'Valentin',
 'apellido': 'Ferrando',
 'cedula': 52953201,
 'ingreso': 2022,
 'uc_aprobadas': ['S1UC1',
  'S1UC2',
  'S1UC3',
  'S1UC4',
  'S1UC5',
  'S2UC1',
  'S2UC2',
  'S2UC3',
  'S2UC4',
  'S3UC1',
  'S3UC2',
  'S3UC3',
  'S3UC4',
  'S3UC5',
  'S4UC1',
  'S4UC2',
  'S4UC4',
  'S4UC5',
  'S5UC1',
  'S5UC2',
  'S5UC3',
  'S5UC4',
  'S6UC1',
  'S6UC2',
  'S6UC3',
  'S6UC4',
  'S6UC5',
  'S6UC6'],
 'uc_matriculadas': ['S7UC2', 'S7UC4'],
 'uc_pendientes': {'examen': {},
  'tutoria': {},
  'examen_unico': {},
  'recursar': []}}

### Prueba agregar/eliminar estudiante a la base de datos de estudiantes desde Sistema (PruebaBasedeDatosEstudiantes.JSON)

In [73]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

sistema= Sistema(ruta, planIBIO)

#sistema.agregar_estudiante("E013", "Samuel", "Vega", 47236551, 2023, uc_aprobadas=["S1UC1", "S1UC2", "S1UC3", "S1UC4", "S1UC5"], uc_matriculadas=["S2UC1", "S2UC2", "S2UC3", "S2UC4"])

#sistema.eliminar_estudiante(47236551)

sistema.info_estudiante("E013")

{'codigo_estudiante': 'E013',
 'nombre': 'Samuel',
 'apellido': 'Vega',
 'cedula': 47236551,
 'ingreso': 2023,
 'uc_aprobadas': ['S1UC1', 'S1UC2', 'S1UC3', 'S1UC4', 'S1UC5'],
 'uc_matriculadas': ['S2UC1', 'S2UC2', 'S2UC3', 'S2UC4'],
 'uc_pendientes': {'examen': {},
  'tutoria': {},
  'examen_unico': {},
  'recursar': []}}

# Clase Estudiante

In [74]:
class Estudiante:
    """ Representa a un estudiante dentro del sistema académico. Esta clase modela los datos personales y académicos del estudiante, incluyendo su historial de unidades curriculares aprobadas, matriculadas y pendientes. """

    def __init__(self, nombre: str, apellido: str, cedula: int, carrera: str, sistema:object, plan:object):
        """Inicializa una nueva instancia de Estudiante a partir de su nombre, apellido, cédula y carrera.

        Parámetros:
            nombre (str): Nombre del estudiante.
            apellido (str): Apellido del estudiante.
            cedula (int): Cédula de identidad del estudiante.
            carrera (str): Carrera a la que pertenece el estudiante."""
        
        self.nombre = nombre
        self.apellido = apellido
        self.__cedula = cedula
        self.carrera = carrera

        self.__plan = plan
        self.__sistema = sistema

        info = self.__sistema.info_estudiante(cedula)
        self.__codigo_estudiante = info["codigo_estudiante"]
        self.__ingreso = info["ingreso"]                        
        self.__uc_aprobadas = info["uc_aprobadas"]              
        self.__uc_matriculadas = info["uc_matriculadas"]        
        self.__uc_pendientes = info["uc_pendientes"]               
    
    @property
    def ver_codigo_estudiante(self) -> str:
        """Devuelve el código único del estudiante."""
        return self.__codigo_estudiante

    @property
    def ver_cedula(self) -> int:
        """Devuelve la cédula de identidad del estudiante."""
        return self.__cedula

    @property
    def ver_ingreso(self) -> int:
        """Devuelve el año de ingreso del estudiante."""
        return self.__ingreso

    @property
    def ver_uc_aprobadas(self) -> list:
        """Devuelve la lista de UC aprobadas por el estudiante."""
        return self.__uc_aprobadas

    @property
    def ver_uc_matriculadas(self) -> list:
        """Devuelve la lista de UC en las que el estudiante está actualmente matriculado."""
        return self.__uc_matriculadas

    @property
    def ver_uc_pendientes(self) -> dict:
        """Devuelve la lista de UC que el estudiante tiene pendientes."""
        return self.__uc_pendientes
    
    def matricularse_uc(self, *codigos_uc: str) -> str:
        """Matricula al estudiante en una o varias UCs.

        Parámetro:
            codigo_uc (str): Código de la UC a la que desea matricular."""
        
        return self.__sistema.matricular_estudiante_UC(self.__codigo_estudiante, *codigos_uc)
    
    def desmatricularse_uc(self, *codigos_uc: str) -> str:
        """Desmatricula al estudiante de una o varias UCs

        Parámetro:
            codigo_uc (str): Código de la UC a la que desea desmatricular."""
        
        return self.__sistema.desmatricular_estudiante_UC(self.__codigo_estudiante, *codigos_uc)

    def inscribirse(self, codigo_uc: str, estado:str) -> str:
        """Matricula al estudiante a un examen, tutoria o examen único.

        Parámetro:
            codigo_uc (str): Código de la UC a la que desea inscribirse.
            estado (str): Nombre del estado ("examen", "tutoria", "examen_unico")."""
       
        return self.__sistema.inscripcion(self.__codigo_estudiante, codigo_uc, estado)

    def ver_previas_uc(self, codigo_uc: str) -> list:
        """Devuelve las previaturas necesarias para una UC según el plan de estudios.

        Parámetro:
            codigo_uc (str): Código de la UC.

        Retorna:
            list: Lista de códigos de UCs previas requeridas."""   
               
        return self.__plan.previas_de_uc(codigo_uc)
    
    def creditos_obtenidos(self) -> int:
        """ """
        
        return self.__plan.sumar_creditos(self.__uc_aprobadas)  
                                                               

### Prueba de la Clase Estudiante

In [75]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

sistema= Sistema(ruta, planIBIO)

Vale= Estudiante("Valentin", "Ferrando", 52953201, "IBIO", datos_estudiantes, planIBIO)

print(Vale.ver_cedula)

print(Vale.ver_uc_aprobadas)

print(Vale.ver_uc_matriculadas)

print(f"La cantidad total de créditos obtenidos son: {Vale.creditos_obtenidos()}")

52953201
['S1UC1', 'S1UC2', 'S1UC3', 'S1UC4', 'S1UC5', 'S2UC1', 'S2UC2', 'S2UC3', 'S2UC4', 'S3UC1', 'S3UC2', 'S3UC3', 'S3UC4', 'S3UC5', 'S4UC1', 'S4UC2', 'S4UC4', 'S4UC5', 'S5UC1', 'S5UC2', 'S5UC3', 'S5UC4', 'S6UC1', 'S6UC2', 'S6UC3', 'S6UC4', 'S6UC5', 'S6UC6']
['S7UC2', 'S7UC4']
La cantidad total de créditos obtenidos son: 237


# Clase Secretaria

In [76]:
class Secretaria:
    """ Representa al rol administrativo de Secretaría dentro del sistema académico. Esta clase permite a la secretaria gestionar inscripciones y calificaciones de estudiantes en unidades curriculares y exámenes."""
    
    def __init__(self, nombre: str, apellido: str, codigo: str, carrera: str, sistema:object, plan:object):
        """Inicializa una nueva secretaria con su nombre y apellido.

        Parámetros:
            nombre (str): Nombre de la secretaria.
            apellido (str): Apellido de la secretaria."""

        self.nombre = nombre
        self.apellido = apellido
        self.carrera = carrera
    
        self.__plan = plan
        self.__sistema = sistema
        
        self.__sistema._verificar_secretaria(nombre, apellido, codigo)


    def matricular_estudiante_uc(self, codigo_estudiante: str, *codigo_uc: str) -> str:
        """Matricula a un estudiante en una unidad curricular.

        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular."""
        
        return self.__sistema.matricular_estudiante_UC(codigo_estudiante, *codigo_uc)

    def desmatricular_estudiante_UC(self, codigo_estudiante: str, *codigo_uc: str) -> str:
        """Desmatricula a un estudiante de una unidad curricular.

        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular."""
        
        return self.__sistema.desmatricular_estudiante_UC(codigo_estudiante, *codigo_uc)

    def inscribir_estudiante(self, codigo_estudiante: str, codigo_uc: str, estado: str) -> str:
        """Inscribe a un estudiante en un modalidad (examen, tutoría o examen único).
        
        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular a la que se desea inscribir.
            estado (str): Modalidad de inscripción ('examen', 'tutoria' o 'examen_unico').

        Retorna:
            str: Mensaje con el resultado de la inscripción (éxito o descripción del error)."""
        
        return self.__sistema.inscripcion(codigo_estudiante, codigo_uc, estado)

    def desinscribir_estudiante(self, codigo_estudiante: str, examen: str) -> str:
        """Desmatricula a un estudiante de un examen.

        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            examen (str): Identificador del examen."""
        
        pass   #FALTA IMPLEMENTAR
    
    def poner_nota_estudiante(self, codigo_estudiante: str, codigo_uc: str, estado:str, nota: float) -> str:
        """Registra la nota obtenida por un estudiante en un estado (examen, tutoría, examen único) y actualiza su situación académica.
        
        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular evaluada.
            estado (str): Modalidad de evaluación ('examen', 'tutoria' o 'examen_unico').
            nota (float): Nota obtenida por el estudiante.

        Retorna:
            str: Mensaje indicando si el registro fue exitoso o describiendo el error."""
            
        return self.__sistema.poner_nota(codigo_estudiante, codigo_uc, estado, nota)

    def lista_inscritos(self, codigo_UC: str, tipo: str = None) -> list:
        """ """
        
        return self.__sistema.obtener_inscritos(codigo_UC, tipo)

### Prueba de la Clase Secretaria

In [77]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

sistema= Sistema(ruta, planIBIO)

Jessi = Secretaria("Jessica", "Arocena", "IBIOSEC", "IBIO", sistema, planIBIO)

Jessi.matricular_estudiante_uc("E011", "S7UC2", "S7UC4")

'[S7UC2] Ya está matriculada. - [S7UC4] Ya está matriculada.'

# Clase Coordinadora

In [78]:
class Coordinadora:
    """ Representa a la Coordinadora académica de la carrera. Esta clase permite gestionar el plan de estudio, estudiantes y unidades curriculares, así como autorizar excepciones y registrar calificaciones. """

    def __init__(self, nombre: str, apellido: str, codigo: str, carrera: str, sistema:object, plan:object):
        """Inicializa una nueva coordinadora con nombre y apellido.

        Parámetros:
            nombre (str): Nombre de la coordinadora.
            apellido (str): Apellido de la coordinadora."""
     
        self.nombre = nombre
        self.apellido = apellido
        self.carrera = carrera
    
        self.__plan = plan
        self.__sistema = sistema
        
        self.__sistema._verificar_coordinadora(nombre, apellido, codigo)

    def lista_inscritos(self, codigo_UC: str, tipo: str = None) -> list:
        """ """
        
        return self.__sistema.obtener_inscritos(codigo_UC, tipo)

    def poner_nota_estudiante(self, codigo_estudiante: str, codigo_uc: str, estado:str, nota: float) -> str:
        """Registra la nota obtenida por un estudiante en un estado (examen, tutoría, examen único) y actualiza su situación académica.
        
        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular evaluada.
            estado (str): Modalidad de evaluación ('examen', 'tutoria' o 'examen_unico').
            nota (float): Nota obtenida por el estudiante.

        Retorna:
            str: Mensaje indicando si el registro fue exitoso o describiendo el error."""
            
        return self.__sistema.poner_nota(codigo_estudiante, codigo_uc, estado, nota)

    def autorizar_excepcion_matricula(self, codigo_estudiante: str, codigo_uc: str) -> str:
        """Autoriza una excepción para que un estudiante se matricule en una unidad curricular
        sin haber cumplido las previaturas.

        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular."""
        
        pass   #FALTA IMPLEMENTAR

    def autorizar_excepcion_desmatricula(self, codigo_estudiante: str, codigo_uc: str) -> str:
        """Autoriza una excepción para que un estudiante se desmatricule de una unidad curricular.

        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            codigo_uc (str): Código de la unidad curricular."""        
        
        pass   #FALTA IMPLEMENTAR

    def agregar_estudiante(self, codigo_estudiante: str, nombre_estudiante: str, apellido_estudiante: str, cedula_estudiante: int, ingreso_estudiante: int, uc_aprobadas: list = [], uc_matriculadas: list = []) -> str:
        """Agrega un nuevo estudiante al sistema.

        Parámetros:
            codigo_estudiante (str): Código del estudiante.
            nombre_est (str): Nombre del estudiante.
            apellido_est (str): Apellido del estudiante.
            cedula (int): Cédula de identidad.
            ingreso (int): Año de ingreso.
            uc_aprobadas (list, opcional): Lista de unidades curriculares aprobadas.
            uc_matriculadas (list, opcional): Lista de unidades curriculares en las que está inscripto.
            uc_pendientes (dict, opcional): Unidades curriculares pendientes con motivo."""
        
        return self.__sistema.agregar_estudiante(codigo_estudiante, nombre_estudiante, apellido_estudiante, cedula_estudiante, ingreso_estudiante, uc_aprobadas, uc_matriculadas)
    
    def eliminar_estudiante(self, codigo_estudiante: str) -> str:
        """ """
        
        return self.__sistema.eliminar_estudiante(codigo_estudiante)


### Prueba de la Clase Coordinadora

In [79]:
ruta= 'InfoCarreras.json'

planIBIO = PlanEstudio(ruta, "IBIO")

sistema= Sistema(ruta, planIBIO)

Ella = Coordinadora("Ella", "Escandón", "IBIOCOOR", "IBIO", sistema, planIBIO)

#Ella.eliminar_estudiante("E012")

Ella.agregar_estudiante("E012", "Sofía", "de Bérail", 12223334, 2022, uc_aprobadas=['S1UC1', 'S1UC2', 'S1UC3', 'S1UC4', 'S1UC5', 'S2UC1', 'S2UC2', 'S2UC3', 'S2UC4', 'S3UC1', 'S3UC2', 'S3UC3', 'S3UC4', 'S3UC5', 'S4UC1', 'S4UC2', 'S4UC3', 'S4UC4', 'S4UC5', 'S5UC1', 'S5UC2', 'S5UC3', 'S5UC4', 'S6UC1', 'S6UC2', 'S6UC3', 'S6UC4', 'S6UC5', 'S6UC6'], uc_matriculadas=['S7UC1', 'S7UC2', 'S7UC3', 'S7UC4', 'S7UC5'])

'Ya existe un estudiante con código E012. No se agregó.'

# Clase Academico (Gestor de Archivos)

In [80]:
class GestorAcademico:
    
    def __init__(self, ruta_carreras):
        self.__ruta_carreras = ruta_carreras

    def crear_plan_de_estudios(self, carrera):
        return PlanEstudio(self.__ruta_carreras, carrera)
    
    def crear_sistema(self, carrera):
        return Sistema(self.__ruta_carreras, self.crear_plan_de_estudios(carrera))
    
    def crear_estudiante(self, nombre, apellido, cedula, carrera):
        plan = self.crear_plan_de_estudios(carrera)
        sistema = self.crear_sistema(carrera)
        
        return Estudiante(nombre, apellido, cedula, carrera, sistema, plan)

    def crear_secretaria(self, nombre, apellido, codigo, carrera):
        plan = self.crear_plan_de_estudios(carrera)
        sistema = self.crear_sistema(carrera)
        
        return Secretaria(nombre, apellido, codigo, carrera, sistema, plan)
    
    def crear_coordinadora(self, nombre, apellido, codigo, carrera):
        plan = self.crear_plan_de_estudios(carrera)
        sistema = self.crear_sistema(carrera)
        
        return Coordinadora(nombre, apellido, codigo, carrera, sistema, plan)

In [81]:
ruta_carrera= 'InfoCarreras.JSON'

gestor= GestorAcademico(ruta_carrera)

planIBIO= gestor.crear_plan_de_estudios("IBIO")

print(type(planIBIO))

sistemaIBIO= gestor.crear_sistema("IBIO")

print(type(sistemaIBIO))

Vale = gestor.crear_estudiante("Valentin", "Ferrando", 52953201, "IBIO")

print(type(Vale))

Jessi = gestor.crear_secretaria("Jessica", "Arocena", "IBIOSEC", "IBIO")

print(type(Jessi))

Ella = gestor.crear_coordinadora("Ella", "Escandón", "IBIOCOOR", "IBIO")

print(type(Ella))

<class '__main__.PlanEstudio'>
<class '__main__.Sistema'>
<class '__main__.Estudiante'>
<class '__main__.Secretaria'>
<class '__main__.Coordinadora'>
